In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, RandomizedSearchCV

import pandas as pd
import numpy as np
import os
import glob

In [47]:
# Get data
training_folder = os.path.join("..","..","ProcessedData","Training_Data_20231106")

csv_list = glob.glob(os.path.join(training_folder,"*"))
df_list = [pd.read_csv(csv,index_col=0) for csv in csv_list]

# Process data
df_total = pd.concat(df_list)
df_total.reset_index(drop=True)
df_total.replace([np.inf, -np.inf], np.nan, inplace=True)
df_total.dropna(axis=0,inplace=True)
df_total.drop(labels=["label","Filename","Region"],axis=1,inplace=True)

# Split
X = df_total.drop(labels="Labels",axis=1)
y = df_total["Labels"]
print(X.columns)

Index(['area', 'equivalent_diameter', 'orientation', 'major_axis_length',
       'minor_axis_length', 'perimeter', 'min_intensity', 'mean_intensity',
       'max_intensity', 'solidity', 'eccentricity', 'centroid_local-0',
       'centroid_local-1', 'feret_diameter_max', 'moments-0-0', 'moments-0-1',
       'moments-0-2', 'moments-0-3', 'moments-1-0', 'moments-1-1',
       'moments-1-2', 'moments-1-3', 'moments-2-0', 'moments-2-1',
       'moments-2-2', 'moments-2-3', 'moments-3-0', 'moments-3-1',
       'moments-3-2', 'moments-3-3', 'moments_central-0-0',
       'moments_central-0-1', 'moments_central-0-2', 'moments_central-0-3',
       'moments_central-1-0', 'moments_central-1-1', 'moments_central-1-2',
       'moments_central-1-3', 'moments_central-2-0', 'moments_central-2-1',
       'moments_central-2-2', 'moments_central-2-3', 'moments_central-3-0',
       'moments_central-3-1', 'moments_central-3-2', 'moments_central-3-3',
       'moments_hu-0', 'moments_hu-1', 'moments_hu-2', 'mo

In [48]:
# Remove size- and intensity-aware features
features_oi = [
                'solidity',
                'eccentricity',            
                'moments_hu',
                'major_axis_length/minor_axis_length',
                'perimeter/major_axis_length',
                'perimeter/minor_axis_length'
                ]
X = X[
    v for v in X.columns.values 
        if any([v in feature for feature in features_oi])
]

SyntaxError: unexpected EOF while parsing (38406961.py, line 13)

In [ ]:
# Train-test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=.7,stratify=y)


In [ ]:
# Define Model
n_trees = 64
samples_per_leaf = 3
clf = RandomForestClassifier(n_estimators=n_trees,min_samples_leaf=samples_per_leaf,
                             random_state=74)


In [ ]:
rfecv = RFECV(clf,cv=5,scoring="f1_macro",min_features_to_select=5)


In [ ]:
rfecv.fit(X_train,y_train)

RFECV(cv=5,
      estimator=RandomForestClassifier(min_samples_leaf=3, n_estimators=64),
      min_features_to_select=5, scoring='f1_macro')

In [ ]:
print(rfecv.ranking_)
set(X.columns.values)-set(X.columns.values[rfecv.support_])


[ 1  1  1  1  1  1  2  1  1  1  1  1 25  1  1 22 23  7  1 15 20  4  6 26
 13  1 12 18 27 10  1 30  1  3 31 11  1 24 19 17  1  9  1  5 14 16  1  1
  1  1 28  8 29  1  1  1 21  1]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 57 but corresponding boolean dimension is 58